<a href="https://colab.research.google.com/github/guyyariv/AI-Beat-Maker/blob/master/adding_drums.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Imports and installations
#@test {"output": "ignore"}
print('Installing dependencies...')
!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -qU pyfluidsynth pretty_midi

!pip install -qU magenta

# Hack to allow python to pick up the newly-installed fluidsynth lib. 
# This is only needed for the hosted Colab environment.
import ctypes.util
orig_ctypes_util_find_library = ctypes.util.find_library
def proxy_find_library(lib):
  if lib == 'fluidsynth':
    return 'libfluidsynth.so.1'
  else:
    return orig_ctypes_util_find_library(lib)
ctypes.util.find_library = proxy_find_library

print('Importing libraries and defining some helper functions...')
from google.colab import files

import magenta
import note_seq
import tensorflow

print('🎉 Done!')
print(magenta.__version__)
print(tensorflow.__version__)

# ____________________________________________________
!git clone https://github.com/bearpelican/musicautobot.git
import os
os.chdir('musicautobot')
!pip install torch fastai music21 pebble fluidsynth midi2audio
from musicautobot.numpy_encode import *
from musicautobot.utils.file_processing import process_all, process_file
from musicautobot.config import *
from musicautobot.music_transformer import *
from musicautobot.utils.setup_musescore import setup_musescore
setup_musescore()
from midi2audio import FluidSynth
from IPython.display import Audio
# ____________________________________________________
!apt install fluidsynth
!cp /usr/share/sounds/sf2/FluidR3_GM.sf2 ./font.sf2
from IPython.display import Audio
from midi2audio import FluidSynth
from pydub import AudioSegment
# ____________________________________________________

print('Copying checkpoint from GCS. This will take less than a minute...')
# This will download the mel_2bar_big checkpoint. There are more checkpoints that you
# can use with this model, depending on what kind of output you want
# See the list of checkpoints: https://github.com/magenta/magenta/tree/main/magenta/models/music_vae#pre-trained-checkpoints
!gsutil -q -m cp -R gs://download.magenta.tensorflow.org/models/music_vae/colab2/checkpoints/mel_2bar_big.ckpt.* /content/

# Import dependencies.
from magenta.models.music_vae import configs
from magenta.models.music_vae.trained_model import TrainedModel

# Initialize the model.
print("Initializing Music VAE...")
music_vae = TrainedModel(
      configs.CONFIG_MAP['cat-mel_2bar_big'], 
      batch_size=4, 
      checkpoint_dir_or_path='/content/mel_2bar_big.ckpt')

print('🎉 Done!')

# ____________________________________________________

print('Downloading model bundle. This will take less than a minute...')
note_seq.notebook_utils.download_bundle('basic_rnn.mag', '/content/')

# Import dependencies.
from magenta.models.melody_rnn import melody_rnn_sequence_generator
from magenta.models.shared import sequence_generator_bundle
from note_seq.protobuf import generator_pb2
from note_seq.protobuf import music_pb2

# Initialize the model.
print("Initializing Melody RNN...")
bundle = sequence_generator_bundle.read_bundle_file('/content/basic_rnn.mag')
generator_map = melody_rnn_sequence_generator.get_generator_map()
melody_rnn = generator_map['basic_rnn'](checkpoint=None, bundle=bundle)
melody_rnn.initialize()

print('🎉 Done!')

# ____________________________________________________

from note_seq.protobuf import music_pb2

Installing dependencies...
Selecting previously unselected package fluid-soundfont-gm.
(Reading database ... 155222 files and directories currently installed.)
Preparing to unpack .../fluid-soundfont-gm_3.1-5.1_all.deb ...
Unpacking fluid-soundfont-gm (3.1-5.1) ...
Selecting previously unselected package libfluidsynth1:amd64.
Preparing to unpack .../libfluidsynth1_1.1.9-1_amd64.deb ...
Unpacking libfluidsynth1:amd64 (1.1.9-1) ...
Setting up fluid-soundfont-gm (3.1-5.1) ...
Setting up libfluidsynth1:amd64 (1.1.9-1) ...
Processing triggers for libc-bin (2.27-3ubuntu1.3) ...
/sbin/ldconfig.real: /usr/local/lib/python3.7/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link

     |████████████████████████████████| 5.6 MB 4.7 MB/s 
     |████████████████████████████████| 51 kB 5.2 MB/s 
     |████████████████████████████████| 1.4 MB 4.5 MB/s 
     |████████████████████████████████| 69 kB 7.0 MB/s 
     |████████████████████████████████| 352 kB 66.5 MB/s 
     |███████████████████

/usr/local/lib/python3.7/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/usr/local/lib/python3.7/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


🎉 Done!
2.1.3
2.7.0
Cloning into 'musicautobot'...
remote: Enumerating objects: 4131, done.
remote: Total 4131 (delta 0), reused 0 (delta 0), pack-reused 4131
Receiving objects: 100% (4131/4131), 83.43 MiB | 13.52 MiB/s, done.
Resolving deltas: 100% (2992/2992), done.
  Created wheel for fluidsynth: filename=fluidsynth-0.2-py3-none-any.whl size=4514 sha256=28c0b30d81a1ebfd9e2ea27582d83ebf3949adac26aa05985aaf9dc533bb6432
  Stored in directory: /root/.cache/pip/wheels/44/4b/81/dee2d535b2dd27af13c230ba49aa6f05a5dff0f57c68acef55
Successfully built fluidsynth


Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libqt5x11extras5 qsynth
Suggested packages:
  jackd
The following NEW packages will be installed:
  fluidsynth libqt5x11extras5 qsynth
0 upgraded, 3 newly installed, 0 to remove and 67 not upgraded.
Need to get 220 kB of archives.
After this operation, 855 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fluidsynth amd64 1.1.9-1 [20.7 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libqt5x11extras5 amd64 5.9.5-0ubuntu1 [8,596 B]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 qsynth amd64 0.5.0-2 [191 kB]
Fetched 220 kB in 2s (125 kB/s)
Selecting previously unselected package fluidsynth.
(Reading database ... 155237 files and directories currently installed.)
Preparing to unpack .../fluidsynth_1.1.9-1_amd64.deb ...
Unpacking fluidsynth (1.1.9-1) ...
Selec

/usr/local/lib/python3.7/dist-packages/magenta/models/music_vae/lstm_utils.py:99: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  name=name),
/usr/local/lib/python3.7/dist-packages/keras/legacy_tf_layers/core.py:255: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  return layer.apply(inputs)
/usr/local/lib/python3.7/dist-packages/magenta/contrib/rnn.py:751: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  self._names["W"], [input_size + self._num_units, self._num_units * 4])
/usr/local/lib/python3.7/dist-packages/magenta/contrib/rnn.py:754: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  initializer=tf.constant_initializer(0.0))


Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Do not call `graph_parents`.


/usr/local/lib/python3.7/dist-packages/magenta/models/music_vae/base_model.py:199: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  kernel_initializer=tf.random_normal_initializer(stddev=0.001))
/usr/local/lib/python3.7/dist-packages/magenta/models/music_vae/base_model.py:205: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  kernel_initializer=tf.random_normal_initializer(stddev=0.001))


INFO:tensorflow:Restoring parameters from /content/mel_2bar_big.ckpt
🎉 Done!
Initializing Melody RNN...
'model_variables' collection should be of type 'byte_list', but instead is of type 'node_list'.
INFO:tensorflow:Restoring parameters from /tmp/tmpfmlz7fsc/model.ckpt
🎉 Done!


In [2]:
def get_drum_notes(peak_times, inner_peak_times):
  drums = music_pb2.NoteSequence()

  j = 0
  for p in peak_times:
    pitch = 55
    if j % 2 == 0:
      pitch = 65
    drums.notes.add(pitch=pitch, start_time=p-0.05, end_time=p+0.1, is_drum=True, instrument=10, velocity=100)
    j += 1

  j = 0
  for ip in inner_peak_times:
    pitch = 40
    if j % 2 == 0:
      pitch = 50
    drums.notes.add(pitch=pitch, start_time=ip-0.05, end_time=ip+0.1, is_drum=True, instrument=10, velocity=100)
    j += 1

  drums.total_time = max(peak_times[-1], inner_peak_times[-1])+0.1

  drums.tempos.add(qpm=60)

  # This is a colab utility method that visualizes a NoteSequence.
  note_seq.plot_sequence(drums)

  # This is a colab utility method that plays a NoteSequence.
  note_seq.play_sequence(drums,synth=note_seq.fluidsynth)
  return drums

def convert_notesq_to_midi(note_sq, path):
  note_seq.sequence_proto_to_midi_file(note_sq, path)

def convert_midi_to_wav(inp_path, out_path):
  !fluidsynth -ni font.sf2 inp_path -F out_path -r 44100
  Audio(str(out_path))

def change_wav_volume(wav_path, inc_dec, num, exp_path):
  song = AudioSegment.from_wav(wav_path)
  if inc_dec == "UP":
    song = song + num
  elif inc_dec == "DOWN":
    song = sung - num
  else:
    song = song

  song.export(exp_path, "wav")


def combine_2_wav_files(wav_one, wav_two, exp):
  sound1 = AudioSegment.from_file(wav_one)
  sound2 = AudioSegment.from_file(wav_two)
  combined = sound1.overlay(sound2)
  combined.export(exp, format='wav')

In [3]:
#beat tracking

In [ ]:

drums = get_drum_notes(peaks_times, inner_peaks_times)
convert_notesq_to_midi(drums, '/content/musicautobot/drums_sample_output.mid')
convert_midi_to_wav(/content/musicautobot/drums_sample_output.mid, /content/musicautobot/output_11.wav)
change_wav_volume('/content/musicautobot/output_11.wav', "UP", 18, '/content/musicautobot/louder_11.wav')
Audio('/content/musicautobot/louder_11.wav')
combine_2_wav_files("/content/30_sec_test.wav", "/content/musicautobot/louder_11.wav", "/content/combined.wav")
Audio('/content/combined.wav')
